<a href="https://colab.research.google.com/github/dssg/ai4med/blob/main/explore_emr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring EMR Data

## Getting Set up

Running the code below will create a local postgres 13 database for you and
import the sample data. Don't worry about the details of that and you shouldn't need to touch any of the code here aside from running it. Below, we'll talk about how to access the database from within the notebook to run queries.

In [ ]:
# Install and start postgresql-13 server
!sudo apt-get -y -qq update
!wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -
!echo "deb http://apt.postgresql.org/pub/repos/apt/ `lsb_release -cs`-pgdg main" |sudo tee  /etc/apt/sources.list.d/pgdg.list
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql-13 postgresql-client-13
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres psql -U postgres -c "CREATE USER dssg_admin PASSWORD 'dssg_admin';"
!sudo -u postgres psql -U postgres -c "ALTER USER dssg_admin WITH SUPERUSER;"
!sudo -u postgres psql -U postgres -c "DROP DATABASE IF EXISTS mimic;"
!sudo -u postgres psql -U postgres -c "create database mimic owner dssg_admin;"

# Setup a database with name `mimic` to be used

# Environment variables for connecting to the database
%env DEMO_DATABASE_NAME=mimic
%env DEMO_DATABASE_HOST=localhost
%env DEMO_DATABASE_PORT=5432
%env DEMO_DATABASE_USER=dssg_admin
%env DEMO_DATABASE_PASS=dssg_admin

## Download sample data and load it into our postgresql database server
You might see some warnings or errors but ignore them for now

In [ ]:
# Download sample data and load it into our postgres server
!curl -s -OL https://dsapp-public-data-migrated.s3.us-west-2.amazonaws.com/mimicsample.dump
!PGPASSWORD=$DEMO_DATABASE_PASS pg_restore -h $DEMO_DATABASE_HOST -p $DEMO_DATABASE_PORT -d $DEMO_DATABASE_NAME -U $DEMO_DATABASE_USER -O -j 8 mimicsample.dump

In [ ]:
# Install triage and its dependencies
!pip install triage


# Want other packages?
If you need to install any other python modules for your analysis, you can easily do so from a code block by prefixing your pip install command with an ! character. For instance:

!pip install PyYAML

# **You might need to run the cell below twice if you get an error the first time**

---



In [ ]:
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine
import pandas as pd
pd.set_option('display.max_columns', None)

db_url = URL(
            'postgres',
            host='localhost',
            username='dssg_admin',
            database='mimic',
            password='dssg_admin',
            port=5432,
        )

db_engine = create_engine(db_url)


# Querying the Database

The code block above used the sqlalchemy module to create a connection to the database called db_engine. An easy way to run SQL queries against this database is to use the read_sql command provided by pandas. For instance, if you run the example below, it should return the rows fom the **patients** table:

## Let's look at the patients table

In [ ]:
pd.read_sql('SELECT * FROM mimiciii.patients', db_engine)

## Now we can do the same for other tables in the data
 - admissions
 - icustays
 - ...


## We can load a table into a dataframe in python

In [ ]:
patients = pd.read_sql('SELECT * FROM mimiciii.patients', db_engine)
patients.head()

In [ ]:
# prompt: code to join database tables

import pandas as pd
# Join patients and admissions tables on the subject_id column
joined_table = pd.read_sql('''
SELECT *
FROM mimiciii.patients AS p
INNER JOIN mimiciii.admissions AS a
ON p.subject_id = a.subject_id
''', db_engine)

# Print the first few rows of the joined table
joined_table.head()


In [ ]:
# prompt: plot frequency distribution of admission_type

import matplotlib.pyplot as plt

# Create a bar chart of admission type frequencies
plt.bar(joined_table['admission_type'].value_counts().index, joined_table['admission_type'].value_counts().values)
plt.xlabel('Admission Type')
plt.ylabel('Frequency')
plt.title('Frequency Distribution of Admission Type')
plt.show()


In [ ]:
# prompt: color by expire_flag

import matplotlib.pyplot as plt

# Group the data by admission type and expire_flag
grouped_data = joined_table.groupby(['admission_type', 'expire_flag']).size().unstack()

# Create a bar chart with different colors for each expire_flag
plt.bar(grouped_data.index, grouped_data[0], label='Not Expired', color='blue')
plt.bar(grouped_data.index, grouped_data[1], bottom=grouped_data[0], label='Expired', color='red')

plt.xlabel('Admission Type')
plt.ylabel('Frequency')
plt.title('Frequency Distribution of Admission Type by Expire Flag')
plt.legend()
plt.show()


# Homework Assignment

---



# **Question 1: Diagnoses**

1A. What are the 10 most common diagnoses (names of the diagnosis - not just the code) in this sample data?
Hint: you'll need to look at the **icd_diagnoses** table to get diagnosis codes and the **d_icd_diagnoses** table to get the english descriptions.

1B. Show top 10 most common diagnoses for all genders?

1C. What are the 10 most common diagnoses for those patients that have died versus those that have not? Does the answer make medical sense? why or why not?

**1D (Optional Advanced)**: How does the distribution of  diagnoses vary across different age groups? Identify the most common diagnoses by age group and discuss any notable trends.

Question 2: Length of Stay

2A. Calculate the mean, median minimum, maximum, and standard deviation of length of stay for all patients?

2B. Plot the distribution of length of stay over all patients

2C. What percentage of patients have stays under 4 days?

2D. Compare visits/icu stays that are less thna or equal to 4 days to those that are longer than 4 days. What is the difference between the patient characteristics in each group? what about diagnosis? what about procedures? Prescriptions? Feel free to explore all the data and tables available to you (usiung the documentation) and write a short report on in what ways are those two groups of patients similar and in what ways are they different?

Question 3. What is a clinical question you would want to ask using this data?
 - what is the question
 - what action will you take with the answer you get
 - how could you use AI to help with this use case?